In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
# notebook will reload external python modules
%load_ext autoreload
%autoreload 2

## 分类树的建立与剪枝

In [2]:
# import data
names = ['age','sex','cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', \
         'oldpeak', 'slope', 'ca', 'thal', 'goal']
data_raw = pd.read_csv('./data/processed.cleveland.data', names = names, header=None, na_values=['?'])
# drop rows with null values 
data_used = data_raw.dropna()[['sex','cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', \
        'thal', 'goal']] # or use data_raw.fillna(values) to replace null data with values
data_used.describe()

,sex,cp,fbs,restecg,exang,slope,ca,thal,goal
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,0.676768,3.158249,0.144781,0.996633,0.326599,1.602694,0.676768,4.730640,0.946128
std,0.468500,0.964859,0.352474,0.994914,0.469761,0.618187,0.938965,1.938629,1.234551
min,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,0.000000,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,1.000000,3.000000,0.000000,1.000000,0.000000,2.000000,0.000000,3.000000,0.000000
75%,1.000000,4.000000,0.000000,2.000000,1.000000,2.000000,1.000000,7.000000,2.000000
max,1.000000,4.000000,1.000000,2.000000,1.000000,3.000000,3.000000,7.000000,4.000000


In [3]:
for item in data_used:
    print(item + '\'s unique number:' + str(data_used[item].nunique()))
all_num = data_used.shape[0]
val_num = all_num // 5
test_num = all_num // 5
train_num = all_num - val_num - test_num
print('all_num:{}, val_num:{}, test_num:{}, train_num:{}'.format(all_num, val_num, test_num, train_num))

sex's unique number:2
cp's unique number:4
fbs's unique number:2
restecg's unique number:3
exang's unique number:2
slope's unique number:3
ca's unique number:4
thal's unique number:3
goal's unique number:5
all_num:297, val_num:59, test_num:59, train_num:179


In [4]:
# shuffle the set
data_shuffled = np.array(data_used)
np.random.shuffle(data_shuffled)

# mask
train_mask = range(train_num)
val_mask = range(train_num,train_num+val_num)
test_mask = range(train_num + val_num, train_num + val_num + test_num)
# devide set into 3 parts (train, val, test)
x_train = data_shuffled[train_mask, :-1]
y_train = data_shuffled[train_mask, -1]
x_val = data_shuffled[val_mask, :-1]
y_val = data_shuffled[val_mask, -1]
x_test = data_shuffled[test_mask, :-1]
y_test = data_shuffled[test_mask, -1]

In [5]:
# define binary tree class
class Tree:
    def __init__(self, x, y, feature_index, feature_value):
        self.x = x
        self.y = y
        self.feat_idx = feature_index
        self.feat_val = feature_value
        self.left = None
        self.right = None
    def insert_left(self, set_index, feature_index, feature_value):
        if self.left == None:
            self.left = Tree(self.x[set_index], self.y[set_index], feature_index, feature_value)
        else:
            raise ValueError("The node already had a left child !")
    def insert_right(self, set_index, feature_index, feature_value):
        if self.right == None:
            self.right = Tree(self.x[set_index], self.y[set_index], feature_index, feature_value)
        else:
            raise ValueError("The node already had a right child !")
    def get_left(self):
        if self.left != None:
            return self.left
        else:
            raise ValueError("Chile doesn't exist")
    def get_right(self):
        if self.right != None:
            return self.right
        else:
            raise ValueError("Chile doesn't exist")
    def get_info(self):
        return self.x, self.y, self.feat_idx, self.feat_val

In [6]:
def check_accuracy(Y, fx):
    num_correct = np.sum(Y == fx)
    num_samples = len(Y)
    acc = float(num_correct) / num_samples
    print('Got {} / {} correct {:.2%}'.format(num_correct, num_samples, acc))

In [8]:
# Classcification tree
class DT_CT():
    def __init__(self, threshold_num = 2, threshold_gini = 0.2):
        '''
        threshold_num:若该节点的数据集样本数量少于该值，则结束
        threshold_gini:若该节点的基尼指数小于该值，则结束
        '''
        self.root = None # root of the DT
        self.N = len(Y) # number of rows
        self.A = X.shape[1] # number of attributes
        self.features = None
        self.thld_num = threshold_num
        self.thld_gini = threshold_gini
    def collect_feature(self, X):
        '''
        X: data with all avaliable features
        '''
        self.features = []
        for idx_f in range(X.shape[1]):
            self.features.append(np.unique(X[:,idx_f]))
    def train(self, x, y):
        '''
        x:x_train
        y:y_train
        '''
        set_index = np.arange(x.shape[0])
        self.root = Tree(x, y, -1, -1)
        self._divide_node(self.root)
    def _divide_node(self, root):
        lchild_idx, rchild_idx, feat_idx, feat_thld = self._cal_child(root)
        if feat_idx != -1:
            # Doesn't achieve the goal. Start dividing
            root.insert_left(lchild_idx, feat_idx, feat_thld)
            root.insert_right(rchild_idx, feat_idx, feat_thld)
            self._divide_node(root.get_left())
            self._divide_node(root.get_right())
        else:
            # Stop dividing
            pass
    def _cal_child(self, root):
        x, y, _, _ = root.get_info()
        flag = 0
        if self._gini(y) < self.thld_gini:
            flag = 1
        elif len(x) < self.thld_num:
            flag = 1
        # return if achieve the goals
        if flag = 1:
            return None, None, -1, -1
        else:
            pass
        # go on
        feat_num = self.A
        # Iterate every feature and its choices
        for feat_idx in range(feat_num):
            feat_choices = self.features[feat_idx]
            min_gini = 1
            min_feat_idx = None
            min_feat = None
            for feat in feat_choices:
                d1_idx = np.arange(len(x))[x[feat_idx] == feat]
                d2_idx = np.arange(len(x))[x[feat_idx] != feat]
                if d1_idx + d2_idx != len(x):
                    raise ValueError("d1_idx, d2_idx error")
                gini = (self._gini(y[d1_idx]) * len(d1_idx) \
                    + self._gini(y[d2_idx]) * len(d2_idx)) / len(x)
                # Choose the smaller value
                if gini < min_gini:
                    min_gini = gini
                    min_feat_idx = feat_idx
                    min_feat = feat
        # Dividing the node by the smallest keys 
        lchild_idx = np.arange(len(x))[x[min_feat_idx] == min_feat]
        rchild_idx = np.arange(len(x))[x[min_feat_idx] != min_feat]
        return lchild_idx, rchild_idx, min_feat_idx, min_feat
    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        gini = 1
        for item in counts:
            gini -= (item / len(y))**2
        return gini
    def pred(self, x):
        preds = []
        for item in x:
            pass
        return preds

In [9]:
model = NB_mdf(data_shuffled[:,:-1], data_shuffled[:,-1])
# model.train(data_shuffled[:,:-1], data_shuffled[:,-1])
model.train(x_train, y_train)
preds = model.pred(x_val)
check_accuracy(y_val, preds)

Got 29 / 59 correct 49.15%


In [10]:
preds = model.pred(x_test)
check_accuracy(y_test, preds)

Got 31 / 59 correct 52.54%


感觉正确率挺低的所以用sklearn验证一下

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
clf_preds = clf.predict(x_val)
check_accuracy(y_val, clf_preds)
clf_preds = clf.predict(x_test)
check_accuracy(y_test, clf_preds)

Got 30 / 59 correct 50.85%
Got 27 / 59 correct 45.76%
